In [1]:
import transformers

In [2]:
transformers.logging.set_verbosity_info

<function transformers.utils.logging.set_verbosity_info()>

In [3]:
import re
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#hide_output
from transformers import TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

**언어모델 및 Tokenizer 불러오기**

**데이터셋 가져오기**

In [27]:
data_path = "augmented_train.csv"
data_2_path = "general_4000.csv"
data = pd.read_csv(data_path)
data_2 = pd.read_csv(data_2_path)
print(data.head())
print(data_2.head())

   idx      class                                       conversation
0  0.0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1  1.0      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2  2.0  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3  3.0      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4  4.0      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
   Unnamed: 0  class                                       conversation
0           0  일반 대화  맞아 뭐하냐고 똑바로 놓으라고 ㅋㅋ 개웃겨ㅋㅋ 그러다가 잘 못 봐서 칠 때가 가장 ...
1           1  일반 대화  계속 연애하는 것도 있잖아음 그렇긴 하네/그래도 법적으로 이어지지 않은 사이는 불안...
2           2  일반 대화                 응응 하하 마블 역시 영화 잘만드네ㅋㅋ 마자 이터널스도 빨리!
3           3  일반 대화     담 주에 놀러 가면 애들이랑 게임하기로 했어아 놀러 가는구나/무슨 게임하기로 했어?
4           4  일반 대화  재무팀에 가면 뭐가 좋아?음 그래도 좀 더 머리 쓰고 전문적이라서 경리보단 나중에 ...


In [28]:
data.drop('idx', axis = 1, inplace= True)
data_2.drop('Unnamed: 0', axis = 1, inplace= True)

In [32]:
data_all = pd.concat([data, data_2], axis = 0)
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31650 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class         31650 non-null  object
 1   conversation  31649 non-null  object
dtypes: object(2)
memory usage: 741.8+ KB


In [33]:
# class 대화 라벨링하기
data_all['class'].unique()

class_labels = {"협박 대화":0, 
                "갈취 대화":1, 
                "직장 내 괴롭힘 대화": 2, 
                "기타 괴롭힘 대화":3, 
                "일반 대화": 4}

data_all['label'] = data_all['class'].map(class_labels)

In [34]:
data_all.head()

,class,conversation,label
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1


In [35]:
len(data_all)

31650

In [13]:
# new_data_path = os.path.join(os.getenv('HOME'), "aiffel/dktc/test (2).csv")
# new_data = pd.read_csv(new_data_path)

In [14]:
# new_data.head()

In [15]:
# test_data = new_data['text'].tolist()

In [36]:
sampled_data = data_all
sampled_data['conversation'] = sampled_data['conversation'].str.replace('\n', " ")

In [37]:
sampled_data['conversation'] = sampled_data['conversation'].str.replace('nan', "")

In [38]:
nan_rows = sampled_data[sampled_data['conversation'].isna()].index.tolist()
print(f"Rows with nan: {nan_rows}")

Rows with nan: [2468]


In [39]:
sampled_data.iloc[2468]

class                                                       갈취 대화
conversation    나 20만원만 빌려줘라 돈 없는데 필요해서 그래 빌려줘. 갚을게 진짜 돈 없다니까?...
label                                                           1
Name: 2468, dtype: object

In [40]:
sampled_data = sampled_data.drop(sampled_data.index[2468])
len(sampled_data)

31648

In [42]:
sampled_data['label'].value_counts()

3    7658
1    6866
2    6853
0    6272
4    3999
Name: label, dtype: int64

In [43]:
#결측치 확인
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31648 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class         31648 non-null  object
 1   conversation  31648 non-null  object
 2   label         31648 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 989.0+ KB


In [35]:
#중복값 제거
#sampled_data.drop_duplicates(subset = ['conversation'], inpla
# ce = True)

In [44]:
# def remove_null_conversations(dataframe):
#     # 'conversation_split' 열의 null 값 찾기
#     null_conversations = dataframe[dataframe['conversation_split'].isnull()]

#     # null 값이 있는지 확인하고, 있다면 해당 행 제거
#     if not null_conversations.empty:
#         dataframe = dataframe.dropna(subset=['conversation_split'])

#     return dataframe

# 함수 호출
cleaned_data = sampled_data

# # 결과 확인
# cleaned_data['label'].value_counts()

In [45]:
nan_rows = cleaned_data[cleaned_data['conversation'].isna()].index.tolist()
nan_rows

[]

In [49]:
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()

In [50]:
print(type(X_train_list))

<class 'list'>


In [57]:
from transformers import pipeline

In [58]:
classifier = pipeline("sentiment-analysis") 

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [63]:
conversation_list = data_2['conversation'].tolist()
conversation_list

['맞아 뭐하냐고 똑바로 놓으라고 ㅋㅋ 개웃겨ㅋㅋ 그러다가 잘 못 봐서 칠 때가 가장 웃기다고 ㅋㅋ',
 '계속 연애하는 것도 있잖아음 그렇긴 하네/그래도 법적으로 이어지지 않은 사이는 불안할 것 같아',
 '응응 하하 마블 역시 영화 잘만드네ㅋㅋ 마자 이터널스도 빨리!',
 '담 주에 놀러 가면 애들이랑 게임하기로 했어아 놀러 가는구나/무슨 게임하기로 했어?',
 '재무팀에 가면 뭐가 좋아?음 그래도 좀 더 머리 쓰고 전문적이라서 경리보단 나중에 경력에 좋지!',
 '맞아 ㅠㅠ 요즘 아침마다 옷 뭐 입지 고민하다가 지각할뻔해.날씨가 추우니까 출근길이 더 힘들어.',
 '집에서는  왜 그리 절대 안 될꼬 ㅋㅋ집에서 머리 같으면 반나절 지나면 정내 남',
 '너 머리 했다고 했지?응 새로 잘랐어',
 '응 그냥 앉아있다가 말 나와야 가거든...그럼 할일 없어도 앉아있어야 되는거야?...',
 '춘천하면 닭갈비가 제일 유명한데 나는 떡볶이가 맛있었어너는 떡볶이를 좋아해서 그런가봐',
 '그래도 부동산 몇 곳 발품 팔면 잘될 거야 하하맞아 날 추워지기 전에 많은 발품 팔아야지',
 '따뜻해서 좋을 것 같아.나도 또 여행 가고 싶다.',
 '일단 너무 진해져서 송충이 같을까봐 조금 소심한 게 그리는 것 같아그럼 지금은 어떤 제품을 사용중이야?',
 '내일 상견레한다...음식점은 어디로 예약했어?',
 '근처에 게임방 아는 곳 있어?나는 주로 건대입구에 있는 게임방으로 가.',
 '언니 재난지원금 얼마나 남았어?나는 예전에 다 썼지.',
 '너는 언제쯤 결혼 하고 싶어?나는 생각해보면 20대 후반에?',
 '병원에 사람이 정말 많았어난 요즘 소화가 잘 안돼서 걱정이야 ㅠㅠ',
 '괌은 비행기 타고 나가지?응~  투명하고 이쁘더라 ㅜㅜ/비행기 타야 대 하하',
 '땀 엄청 흘리고 노폐물 빠지니까그런 게 있긴 한 가봄 ㅋㅋ 신기해 운동 뭐 하는데',
 '고데기 하기 힘들어 짧은 머리는이것도 괜찮은 거 같아서',
 '나 머리 감을 때랑 말릴 때 빼고는 잘 안만져그렇구나 그리

In [64]:
def clean_conversation_list(conversation_list):
    # NaN 값을 제외한 문자열만 필터링
    cleaned_list = [text for text in conversation_list if isinstance(text, str)]
    
    # 불필요한 문자 제거 함수 정의
    def clean_text(text):
        return text.replace('\n', ' ').replace('/', ' ').strip()
    
    # 모든 대화에 대해 불필요한 문자 제거
    cleaned_list = [clean_text(text) for text in cleaned_list]
    
    return cleaned_list

cleaned_conversation_list = clean_conversation_list(conversation_list)

print(cleaned_conversation_list)

['맞아 뭐하냐고 똑바로 놓으라고 ㅋㅋ 개웃겨ㅋㅋ 그러다가 잘 못 봐서 칠 때가 가장 웃기다고 ㅋㅋ', '계속 연애하는 것도 있잖아음 그렇긴 하네 그래도 법적으로 이어지지 않은 사이는 불안할 것 같아', '응응 하하 마블 역시 영화 잘만드네ㅋㅋ 마자 이터널스도 빨리!', '담 주에 놀러 가면 애들이랑 게임하기로 했어아 놀러 가는구나 무슨 게임하기로 했어?', '재무팀에 가면 뭐가 좋아?음 그래도 좀 더 머리 쓰고 전문적이라서 경리보단 나중에 경력에 좋지!', '맞아 ㅠㅠ 요즘 아침마다 옷 뭐 입지 고민하다가 지각할뻔해.날씨가 추우니까 출근길이 더 힘들어.', '집에서는  왜 그리 절대 안 될꼬 ㅋㅋ집에서 머리 같으면 반나절 지나면 정내 남', '너 머리 했다고 했지?응 새로 잘랐어', '응 그냥 앉아있다가 말 나와야 가거든...그럼 할일 없어도 앉아있어야 되는거야?...', '춘천하면 닭갈비가 제일 유명한데 나는 떡볶이가 맛있었어너는 떡볶이를 좋아해서 그런가봐', '그래도 부동산 몇 곳 발품 팔면 잘될 거야 하하맞아 날 추워지기 전에 많은 발품 팔아야지', '따뜻해서 좋을 것 같아.나도 또 여행 가고 싶다.', '일단 너무 진해져서 송충이 같을까봐 조금 소심한 게 그리는 것 같아그럼 지금은 어떤 제품을 사용중이야?', '내일 상견레한다...음식점은 어디로 예약했어?', '근처에 게임방 아는 곳 있어?나는 주로 건대입구에 있는 게임방으로 가.', '언니 재난지원금 얼마나 남았어?나는 예전에 다 썼지.', '너는 언제쯤 결혼 하고 싶어?나는 생각해보면 20대 후반에?', '병원에 사람이 정말 많았어난 요즘 소화가 잘 안돼서 걱정이야 ㅠㅠ', '괌은 비행기 타고 나가지?응~  투명하고 이쁘더라 ㅜㅜ 비행기 타야 대 하하', '땀 엄청 흘리고 노폐물 빠지니까그런 게 있긴 한 가봄 ㅋㅋ 신기해 운동 뭐 하는데', '고데기 하기 힘들어 짧은 머리는이것도 괜찮은 거 같아서', '나 머리 감을 때랑 말릴 때 빼고는 잘 안만져그렇구나 그리고 머리도 꽉 묶지 말고', '옛날에 

In [65]:
classifier(cleaned_conversation_list)

[{'label': 'NEGATIVE', 'score': 0.5679274797439575},
 {'label': 'POSITIVE', 'score': 0.7940861582756042},
 {'label': 'POSITIVE', 'score': 0.932086706161499},
 {'label': 'NEGATIVE', 'score': 0.5512037873268127},
 {'label': 'POSITIVE', 'score': 0.7458707690238953},
 {'label': 'POSITIVE', 'score': 0.6536434888839722},
 {'label': 'POSITIVE', 'score': 0.7427518367767334},
 {'label': 'NEGATIVE', 'score': 0.7316568493843079},
 {'label': 'NEGATIVE', 'score': 0.6130949854850769},
 {'label': 'POSITIVE', 'score': 0.7891885042190552},
 {'label': 'POSITIVE', 'score': 0.545606255531311},
 {'label': 'POSITIVE', 'score': 0.5991569757461548},
 {'label': 'POSITIVE', 'score': 0.6516026854515076},
 {'label': 'POSITIVE', 'score': 0.6123746633529663},
 {'label': 'POSITIVE', 'score': 0.6370322704315186},
 {'label': 'POSITIVE', 'score': 0.5991659760475159},
 {'label': 'NEGATIVE', 'score': 0.6179830431938171},
 {'label': 'POSITIVE', 'score': 0.7873458862304688},
 {'label': 'POSITIVE', 'score': 0.55326789617538

In [66]:
result = classifier(cleaned_conversation_list)

In [68]:
negative_count = sum(1 for result in result if result['label'] == 'NEGATIVE')
positive_count = sum(1 for result in result if result['label'] == 'POSITIVE')

In [69]:
negative_count

998

In [77]:
import matplotlib.pyplot as plt

In [70]:
positive_count

3001

In [72]:
positive_conversations = [text for text, result in zip(cleaned_conversation_list, result) if result['label'] == 'POSITIVE']

In [73]:
positive = pd.

['계속 연애하는 것도 있잖아음 그렇긴 하네 그래도 법적으로 이어지지 않은 사이는 불안할 것 같아',
 '응응 하하 마블 역시 영화 잘만드네ㅋㅋ 마자 이터널스도 빨리!',
 '재무팀에 가면 뭐가 좋아?음 그래도 좀 더 머리 쓰고 전문적이라서 경리보단 나중에 경력에 좋지!',
 '맞아 ㅠㅠ 요즘 아침마다 옷 뭐 입지 고민하다가 지각할뻔해.날씨가 추우니까 출근길이 더 힘들어.',
 '집에서는  왜 그리 절대 안 될꼬 ㅋㅋ집에서 머리 같으면 반나절 지나면 정내 남',
 '춘천하면 닭갈비가 제일 유명한데 나는 떡볶이가 맛있었어너는 떡볶이를 좋아해서 그런가봐',
 '그래도 부동산 몇 곳 발품 팔면 잘될 거야 하하맞아 날 추워지기 전에 많은 발품 팔아야지',
 '따뜻해서 좋을 것 같아.나도 또 여행 가고 싶다.',
 '일단 너무 진해져서 송충이 같을까봐 조금 소심한 게 그리는 것 같아그럼 지금은 어떤 제품을 사용중이야?',
 '내일 상견레한다...음식점은 어디로 예약했어?',
 '근처에 게임방 아는 곳 있어?나는 주로 건대입구에 있는 게임방으로 가.',
 '언니 재난지원금 얼마나 남았어?나는 예전에 다 썼지.',
 '병원에 사람이 정말 많았어난 요즘 소화가 잘 안돼서 걱정이야 ㅠㅠ',
 '괌은 비행기 타고 나가지?응~  투명하고 이쁘더라 ㅜㅜ 비행기 타야 대 하하',
 '땀 엄청 흘리고 노폐물 빠지니까그런 게 있긴 한 가봄 ㅋㅋ 신기해 운동 뭐 하는데',
 '고데기 하기 힘들어 짧은 머리는이것도 괜찮은 거 같아서',
 '나 머리 감을 때랑 말릴 때 빼고는 잘 안만져그렇구나 그리고 머리도 꽉 묶지 말고',
 '옛날에 앵무새 키우는 사람 신기했는데 ㅋㅋ아 맞아 우리 옆 아파트 살던 오빠도 앵무새 키움 ㅋㅋ',
 '아니 내가 통신사 할인이 되거든오 뭐 공짜로 끊을 수 있고 그래?',
 '그래도 소리는 좀 들어오더라.공사 초기에는 좀더 시끄럽던데 한지 얼마 안됐어?',
 '그리고 모구모구 외에 사이다도 자주 드셔서 제한을 걸어야겠어.당 섭취를 줄이셔야겠다.',
 '선선하니 야외

In [76]:
df_positive = pd.DataFrame({
    "class": ["일반 대화"] * len(positive_conversations),
    "conversation": positive_conversations
})

# CSV 파일로 저장
df_positive.to_csv('positive_conversations.csv', index=False)
